# Análisis Exploratorio de Datos

## Introducción

Uno de los primeros pasos a la hora de realizar un proyecto que involucre el análisis de datos es explorar y visualizar los datos. El objetivo principal es obtener información sobre el contenido de los datos, ayudar a enmarcar las preguntas que haremos y detectar posibles vías para avanzar en las respuestas a estas preguntas.

Trabajaremos con un conjunto de datos clásico para Machine Learning, que consiste en datos inmobiliarios para distritos en el estado de California, EE. UU. De hecho, usaremos una versión ligeramente modificada, preparada por Aurélien Géron.

**Plan**

La idea para hoy y para el jueves es dar los primeros pasos en la lista que hemos visto:

1. Obtención de los datos.

2. Exploración y visualización de los datos para obtener información.
     * ¿Qué tipo de preguntas podemos abordar con este conjunto de datos?

     * ¿Necesitamos datos adicionales?


3. Definición del proyecto.

4. Preprocesamiento. Preparación de los datos para los algoritmos de Machine Learning (El **jueves**).

5. Selección de modelo, entrenamiento, puesta a punto, ... (**Jueves**).

## Celdas preparatorias

Antes que nada, corramos algunas celdas de código para prepararnos. Mucho de lo que viene a continuación está sacado del libro de Aurélien Geron, y su [repo de GitHub](https://github.com/ageron/handson-ml2), que recomendamos.

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "01_AED"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

## Obtención de los datos

El primer paso es hacerse con el conjunto de datos. Un archivo `csv` está disponible en el repositorio del curso, así que simplemente definimos la ruta desde la cual leeremos el archivo.

In [ ]:
HOUSING_PATH = "datasets"

Sin embargo, si estás corriendo desde Colab, necesitamos descargar el archivo específicamente. Para evitar complicaciones al autenticarnos en nuestro repositorio, lo descargaremos de un repositorio público que hicimos para esto. (**Nota**: la siguiente celda no se ejecutará, a propósito, si no está usando Colab).

In [ ]:
if 'google.colab' in sys.modules:
        
    import tarfile

#     DOWNLOAD_ROOT = "https://github.com/ageron/handson-ml2/raw/master/"
#     HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"
    DOWNLOAD_ROOT = "https://github.com/IAI-UNSAM/datasets/raw/master/"
    HOUSING_URL = DOWNLOAD_ROOT + "housing/housing.tgz"

    def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
        os.makedirs(housing_path, exist_ok=True)
        !wget {housing_url} -P {housing_path}
        tgz_path = os.path.join(housing_path, "housing.tgz")
        housing_tgz = tarfile.open(tgz_path)
        housing_tgz.extractall(path=housing_path)
        housing_tgz.close()

    # Corramos la función
    fetch_housing_data()

else: 
    print("No estás en Colab. Esta celda no hizo nada.")

In [ ]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

# The function loads the data as a Pandas DataFrame instance.
housing = load_housing_data()

In [ ]:
housing.head?

## Exploración rápida de los datos

Ahora tenemos los datos en un formato muy práctico (y de uso común), una instancia de `pandas.DataFrame`.

Si no estás familiarizado con `Pandas`, veremos formas muy básicas de ver el contenido de una tabla. Acordate que siempre se puede agregar "?" A un objeto o método para acceder a su documentación (por ejemplo, `housing?` o `housing.head?`).

***

**1.** El método `.info` da información básica del contenido de cada columna.

In [ ]:
housing.info()

Vemos que hay **20640** entradas, con 10 columnas cada una. A excepción de `ocean_proximity`, todas las columnas son números (`float64`). Además, faltan algunas entradas en la columna `total_bedrooms`.

Cada una de estas filas corresponde a un distrito de California. En la analogía del ejemplo de las acciones, cada fila es un momento en el tiempo en el que se evalúa el precio de las acciones de *Tech1*.

***

**2.** Para ver las primeras líneas de la tabla, se usa el método `head`, que acepta un argumento optional para indicar cuántas líneas mostrar.

In [ ]:
housing.head(10)

**Nota**: se puede acceder al nombre de las columnas con el atributo de `columns`.

In [ ]:
print(housing.columns)

Veamos un poco más de cerca `ocean_proximity`. ¿Qué valores toma? (**Nota**: se puede acceder a las columnas como atributos del `DataFrame` o con una sintaxis similar a la de un diccionario. En otras palabras, los dos comandos de la celda siguiente son equivalentes).

In [ ]:
print(housing['ocean_proximity'].unique())
print(housing.ocean_proximity.unique())

In [ ]:
type(housing.housing_median_age)

Podemos dar un paso más y ver cuántas veces aparece cada valor.

In [ ]:
print(housing.ocean_proximity.value_counts())

In [ ]:
# Or a normalized version
print(housing.ocean_proximity.value_counts(normalize=True))

***

**3.** Para las columnas numéricas, podemos obtener estadísticos descriptivos simples usando el método `.describe`.

In [ ]:
housing.describe()

 **Nota 1**: este método excluye automáticamente las entradas vacías o `NaN` para el cálculo. Vean la entrada `count` de la columna `total_bedrooms`.
 
 **Nota 2**: se puede ajustar qué percentiles se calculan usando el argumento `percentiles` del método `describe`.

***
**4.** Podemos ordenar la tabla para ver los distritos en los que alguna columna toma valores extremos.

In [ ]:
housing.sort_values(by='median_income', ascending=False)

Tal vez ya identificaron algunos comportamientos patológicos, pero nos guardamos un análisis más detallado para que se diviertan el jueves.

***

**5.** Podemos agrupar entradas en función del valor de una variable determinada. Esto es útil para un análisis rápido de los datos.

Por ejemplo, veamos cómo cambian los valores medios de las columnas con la variable `ocean_proximity`.

In [ ]:
housing.groupby(by='ocean_proximity').mean()

***

**6.** Por último, pero no por eso menos importante, podemos calcular estadísticos que involucran más de una variable. El más común es el coeficiente de correlación de Pearson.

Todavía no estamos para dar una definición formal, pero digamos que el coeficiente de Pearson de dos variables $X$ e $Y$, que llamamos $\hat{\rho_{XY}}$, es un _estimador_ del coeficiente de correlación poblacional:

$$
\hat{\rho_{XY}} = r = \frac{\hat{\mathrm{cov}}_{XY}}{\hat{\sigma}_X \hat{\sigma}_Y}\;\;,
$$
donde 

$$
\hat{\sigma}_X^2 = \frac{1}{N - 1}\sum_{i=1}^N (x_i - \bar{X})^2\;\;,
$$
$$
\hat{\mathrm{cov}}_{XY} = \frac{1}{N - 1}\sum_{i=1}^N (x_i - \bar{X})(y_i - \bar{Y})\;\;,
$$
y
$$
\hat{\mu_X} = \bar{X}\;\;.
$$

Los valores extremos son -1 y 1, para una (anti-)correlación lineal perfecta entre ambas variables.

Ah, entonces $r=0$ significa que las variables no están correlacionadas, ¿no? .... **¡No!**.

Pero no te preocupes por todos estos detalles si esto no te resulta familiar. Para obtener una idea más intuitiva, analicemos esta imagen, tomada de [Wikipedia](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient):

<img src="https://upload.wikimedia.org/wikipedia/commons/d/d4/Correlation_examples2.svg">

Los números sobre cada subimagen indican el valor del coeficiente de Pearson. **¿Qué te hace pensar esta figura?**

Calculemos ahora el coeficiente de Pearson para cada par de variables del conjunto de datos de California.

In [ ]:
# The correlation between all pairs of variables is easily computed with Pandas.
corr_matrix = housing.corr()

In [ ]:
# What size do you expect `corr_matrix` to be?
print(corr_matrix.shape)

Podemos imprimir la matriz; pero más interesante, la podemos graficar.

In [ ]:
print(corr_matrix)

In [ ]:
plt.imshow(corr_matrix)

# Set ticks
xt = plt.xticks(np.arange(9), housing.columns[:-1], rotation=45, ha='right', va='top')
yt = plt.yticks(np.arange(9), housing.columns[:-1], rotation=0, ha='right', va='center')

# Set colorbar
plt.colorbar(label='Pearson CC')

## Preguntas

***
Ahora que recorrimos un poco los datos, podemos deternenos un minuto a hacernos unas preguntas. 
No hay respuestas correctas o incorrectas

1) ¿Qué preguntas pueden responder estos datos?

2) ¿Cuáles serían los primeros pasos para encontrar respuestas a esas preguntas?

3) ¿Qué variables serían más relevantes en ese caso?

***
<font size=5>**¿Descanso?**</font>

## Visualización

¡Una excelente manera de obtener información a partir del conjunto de datos es hacer gráficos!

`Pandas` tiene un lindo *wrapper* a `matplotlib.pyplot`, but intentaremos usar exclusivamente las funciones de `pyplot` (que importamos arriba como `plt`).

Algunas funciones que podemos probar son:

* `plt.plot` o `plt.scatter` para graficar una variable en función de otra.
* `plt.hist` o `plt.bar` para ver cómo se distribuyen los valores de una variable.

Y para los que tengan más tendencia a la estadística:

* `plt.boxplot` para comparar las distribuciones de las variables.
* `plt.violinplot` para hacer lo mismo usando una estimación de *kernel* (que es un algoritmo considerado de machine learning).

Recuerdá que podés obtener la documentación de cada función agregando `?` al nombre. Por ejemplo:

In [ ]:
plt.plot?

Para ponerlo a prueba, hagamos un gráfico simple de latitud versus longitud.

In [ ]:
# Let's make  simple plot of latitude vs longitude
scatter = plt.scatter(housing.longitude, housing.latitude)
#plt.xlabel('Longitude')
#plt.ylabel('Latitude')

Esta es un gráfico bastante feo. No nos dice mucho. Podemos distinguir la forma de California, pero eso es todo.

<img src="images/01_AED/Map_of_California.png" width=350/>

¡Seguro que podemos hacer algo mejor! Estoy seguro de que podés ayudar ...

***
<font size=7>¡Preparen sus teclados!</font>
<!-- ### Prepare your keyboards! -->


* Intentá hacer histogramas de varias variables (las que te parezcan más relevantes). ¿Notás algo que te llame la atención?
* Probá hacer algunos *scatter plots*.
* Se puede incluir más variables en un *scatter plot* usando los argumentos `color` (o `c`) y `size` (o `s`). También, podés intentar obtener gráficos más agradables y útiles haciendo que los puntos sean semi-transparentes con el argumento `alpha`.

**Nota**: podés salvar las figuras que quieras con la función `save_fig`, que definimos al principio del notebook.

In [ ]:
plt.scatter(housing.longitude, housing.latitude)
# save_fig("bad_visualization_plot")

## Redondeando

Para empezar, hagamos el mismo gráfico que arriba usando transparencia para los puntos.

(Para controlar el tamaño de la figura, primero creamos una instancia `figure`)

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
scat = ax.scatter(housing.longitude, housing.latitude, alpha=0.2)
# save_fig("better_visualization_plot")

Ahora podemos ver algunas regiones más densamente pobladas que otras.

*** 

**Pregunta**: ¿qué otras variables incluirías en este plot?

Una opción es incluir `population` y `median_house_value`

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
scat = ax.scatter(housing.longitude, housing.latitude, alpha=0.4, 
                  c=housing.median_income*10, cmap=plt.get_cmap('jet'),
                  s=housing.population/50, label='Population')
plt.colorbar(scat, label='Median income x 10')
leg = ax.legend()
# save_fig("better_visualization_plot")

**Nota**. Usamos un factor de 10 y 50 para el ingreso y la población, respectivamente, para usar correctamente todo el rango de tamaños y colores.

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
scat = ax.scatter(housing.longitude, housing.latitude, alpha=0.2, 
                  c=housing.median_house_value, cmap=plt.get_cmap('jet'),
                  s=housing.median_income*10, label='Income')
plt.colorbar(scat, label='Median house value')
leg = ax.legend()
# save_fig("better_visualization_plot")

Ahora ya tenemos un gráfimo mucho más informativo, que nos puede dar mucha tela para cortar.

Pero podemos hacerlo aun más bonito incuyendo un mapa de California (merci, Géron!)

In [ ]:
if 'google.colab' in sys.modules:
    
    # Download the California image from Geron's repository
    images_path = os.path.join(PROJECT_ROOT_DIR, "images", "01_Intro")
    os.makedirs(images_path, exist_ok=True)
    DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
    filename = "california.png"
    print("Downloading", filename)
    url = DOWNLOAD_ROOT + "images/end_to_end_project/" + filename
    !wget {url} -P {images_path}
    # !mv california.png {images_path}

In [ ]:
import matplotlib.image as mpimg

# Read California image
filename = "california.png"
california_img = mpimg.imread(os.path.join(IMAGES_PATH, filename))


fig = plt.figure(figsize=(12, 9))
plt.scatter(housing.longitude, housing.latitude, s=housing.population/100,
           c=housing.median_house_value, cmap=plt.get_cmap('jet'), label='Population', alpha=0.4)
ax = fig.axes[0]

# Original code using matplotlib wrapper
# ax = housing.plot(kind="scatter", x="longitude", y="latitude", figsize=(10,7),
#                        s=housing['population']/100, label="Population",
#                        c="median_house_value", cmap=plt.get_cmap("jet"),
#                        colorbar=False, alpha=0.4,
#                       )

# Show image below
plt.imshow(california_img, extent=[-124.55, -113.80, 32.45, 42.05], alpha=0.5,
           cmap=plt.get_cmap("jet"))

# Labels
plt.ylabel("Latitude", fontsize=14)
plt.xlabel("Longitude", fontsize=14)

# Tweak colorbar
prices = housing["median_house_value"]
tick_values = np.linspace(prices.min(), prices.max(), 11)
cbar = plt.colorbar(ticks=tick_values/prices.max())
cbar.ax.set_yticklabels(["$%dk"%(round(v/1000)) for v in tick_values], fontsize=14)
cbar.set_label('Median House Value', fontsize=16)

plt.legend(fontsize=16)
# save_fig("california_housing_prices_plot")
plt.show()

Esta gráfica final tiene mucha información, que ahora podemos, además, relacionar con nuestro conocimiento sobre el mercado inmobiliario (es decir, las áreas rojas corresponden a regiones alrededor de Los Ángeles y la Bahía de San Francisco).

__Conclusión__

Partimos de un archivo con columnas separadas por comas, a una visualización llena de información. Para eso, exploramos cómo las variables dependen la una de la otra, y también miramos de cerca peculiaridades de algunas columnas (como valores faltantes). No usamos ni un concepto estadístico (bueno, el coeficiente de Pearson cuenta como uno), ni ningún modelo matemático (todavía).

Este tipo de análisis de datos exploratorio es un primer paso obligatorio en todos los proyectos de aprendizaje automático y una herramienta poderosa para maximizar lo se puede obtener de los datos.